In [4]:
import pandas as pd
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from sklearn.cluster import KMeans, DBSCAN
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import pickle
import os

In [6]:
"""str cleaning should happen here"""
df = pd.read_csv('C:\\Users\\magda\\OneDrive\\Escritorio\\MDS_licitalab\\Modeling\\20231007200103_query_results.csv',
                 encoding='utf-8')#,
                 #nrows=100000)
df['first_two_digits_code'] = df['agilebuyingscode'].apply(lambda x: x[:2])
df['feature_vector']=  df['first_two_digits_code'] + ' '+ df['agileoffereditemsdescripcionofertada']
df['feature_vector'] = df['feature_vector'].apply(lambda x: unidecode(str(x)).lower())
display(df)

,organismosolicitante,agilebuyingscode,agilebuyingsdescription,agilebuyingsstatus,agilebuyingscurrency,agilebuyingsavailableamount,agileitemsname,agileitemsmp_id,agileitemsproductcategory,agileoffereditemsproductoofertado,agileoffereditemscategoriaofertada,agileoffereditemsdescripcionofertada,agileoffereditemspreciototal,taxnumberprovider,provideractivity,adjudicada,first_two_digits_code,feature_vector
0,61.608.502-6,1057501-2964-COT23,SE SOLICITA AGREGAR FICHA TÉCNICA DE LO OFERTA...,OC Emitida,$,1873450.0,Agujas de biopsia,42142504,Equipamiento y suministros médicos / Suministr...,Agujas de biopsia,Equipamiento y suministros médicos / Suministr...,CORCA1815B aguja de biopsia de partes blandas ...,30.0,96.515.660-7,VENTA AL POR MAYOR DE OTROS PRODUCTOS N.C.P.| ...,False,10,10 corca1815b aguja de biopsia de partes bland...
1,61.608.502-6,1057501-2964-COT23,SE SOLICITA AGREGAR FICHA TÉCNICA DE LO OFERTA...,OC Emitida,$,1873450.0,Agujas de biopsia,42142504,Equipamiento y suministros médicos / Suministr...,Agujas de biopsia,Equipamiento y suministros médicos / Suministr...,CORCA1815B aguja de biopsia de partes blandas ...,30.0,96.515.660-7,VENTA AL POR MAYOR DE OTROS PRODUCTOS N.C.P.| ...,False,10,10 corca1815b aguja de biopsia de partes bland...
2,61.608.502-6,1057501-2964-COT23,SE SOLICITA AGREGAR FICHA TÉCNICA DE LO OFERTA...,OC Emitida,$,1873450.0,Agujas de biopsia,42142504,Equipamiento y suministros médicos / Suministr...,Agujas de biopsia,Equipamiento y suministros médicos / Suministr...,CORCA1815B aguja de biopsia de partes blandas ...,30.0,96.515.660-7,VENTA AL POR MAYOR DE OTROS PRODUCTOS N.C.P.| ...,False,10,10 corca1815b aguja de biopsia de partes bland...
3,61.608.502-6,1057501-2964-COT23,SE SOLICITA AGREGAR FICHA TÉCNICA DE LO OFERTA...,OC Emitida,$,1873450.0,Agujas de biopsia,42142504,Equipamiento y suministros médicos / Suministr...,Agujas de biopsia,Equipamiento y suministros médicos / Suministr...,CORCA1815B aguja de biopsia de partes blandas ...,30.0,96.515.660-7,VENTA AL POR MAYOR DE OTROS PRODUCTOS N.C.P.| ...,False,10,10 corca1815b aguja de biopsia de partes bland...
4,70.885.500-6,1208069-35-COT23,INSUMOS DENTALES ESPECIALIDAD ENDODONCIA,OC Emitida,CLP,1156000.0,Cuñas o sets odontológicos,42151663,Equipamiento y suministros médicos / Equipos y...,Cuñas o sets odontológicos,Equipamiento y suministros médicos / Equipos y...,MEDICALTEK CHILE S.A,10680.0,96.696.000-0,"VENTA AL POR MAYOR DE ARTICULOS DE PERFUMERIA,...",True,12,12 medicaltek chile s.a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15301051,69.130.200-8,547641-74-COT23,Los proveedores deberán adjuntar anexo con det...,OC Emitida,CLP,1500000.0,Expendedoras de artículos de higiene,47131702,Equipos y suministros de limpieza / Suministro...,Expendedoras de artículos de higiene,Equipos y suministros de limpieza / Suministro...,"Cotizacion incluye despacho, entrega 1 dia.",1563700.0,77.089.704-1,VENTA AL POR MENOR DE OTROS PRODUCTOS EN COMER...,False,54,"54 cotizacion incluye despacho, entrega 1 dia."
15301052,69.130.200-8,547641-74-COT23,Los proveedores deberán adjuntar anexo con det...,OC Emitida,CLP,1500000.0,Expendedoras de artículos de higiene,47131702,Equipos y suministros de limpieza / Suministro...,Expendedoras de artículos de higiene,Equipos y suministros de limpieza / Suministro...,"Cotizacion incluye despacho, entrega 1 dia.",1563700.0,77.089.704-1,VENTA AL POR MENOR DE OTROS PRODUCTOS EN COMER...,False,54,"54 cotizacion incluye despacho, entrega 1 dia."
15301053,69.130.200-8,547641-74-COT23,Los proveedores deberán adjuntar anexo con det...,OC Emitida,CLP,1500000.0,Expendedoras de artículos de higiene,47131702,Equipos y suministros de limpieza / Suministro...,Expendedoras de artículos de higiene,Equipos y suministros de limpieza / Suministro...,"Cotizacion incluye despacho, entrega 1 dia.",1563700.0,77.089.704-1,VENTA AL POR MENOR DE OTROS PRODUCTOS EN COMER...,False,54,"54 cotizacion incluye despacho, entrega 1 di

In [30]:
class UserVector:
    def __init__(self,taxnumberprovider:str, length:int = 10) -> None:
         
        global df
        
        self.id = taxnumberprovider
        #TODO add randomizer if > 10 descriptions
        self.strings = df.query(f"taxnumberprovider == '{str(taxnumberprovider)}'")['feature_vector'].unique()[:length]
        #TODO revisar que el df no sea nulo
        #if self.df == None:
        #    print('This user has no AgileBuys.')
        self.vector = self.generate_vector()
        #print(self.strings)
    
    def describe(self):
        """ 
        Describes user.
        """
        pass
        
    def generate_vector(self):
        
        """ Concatenates a set of agilebuyingsdescription from taxnumberprovider and vectorizes through CountVectorizer"""
        
        pass
    
class UserSpaceGenerator(UserVector):
    def __init__(self, DF:pd.DataFrame, save_path:str = os.getcwd(), autoinitialize = True,autosave = True) -> None:
        self.DF = DF
        self.vectorizer = CountVectorizer()
        self.corpus,self.qualifying_users = self.generate_corpus()
        self.vectorized_corpus = self.vectorize_corpus()
        if autoinitialize:
            self.tsne_data = self.tsne_reduction()
            self.kmeans_model, self.data_with_clusters = self.launch_kmeans(30,
                                                                            self.tsne_data,
                                                                            self.corpus,
                                                                            plot=True)
        self.save_path = save_path
        if autosave:
            self.export_kmeans_model_and_data()
            self.export_vectorizer()
            self.export_vectorized_corpus()
             
    def generate_corpus(self, n_strings:int = 10, to_csv:bool = False):
        """generates list of UserVector objects from a list of <n_string> taxnumberprovider. 
            

        Args:
            n_strings (int, optional): number of strings to be considered when creating the UserVector objects. Defaults to 10.
            to_csv (bool, optional): whether to export these strings to a csv. Defaults to False.

        Returns:
            _type_: _description_
        """
        gb = self.DF.groupby(by =['taxnumberprovider']).agg({'agilebuyingscode':'nunique'})
        gb = gb.sort_values(by = 'agilebuyingscode')
        qualifying_users =  gb[gb['agilebuyingscode'] >= n_strings].index.values
        print(f'Se han removido {round((gb.shape[0] -qualifying_users.shape[0])/gb.shape[0] *100,2)}% de taxnumberproviders, por tener < {n_strings} licitaciones. \n El numero de usuarios para crear el corpus será {qualifying_users.shape[0]}.')
        
        corpus = [' '.join(UserVector(i).strings) for i in tqdm(qualifying_users, desc = 'Selecting strings from each user')]
         
        #TODO fix path for save
        if to_csv:
            pd.DataFrame(corpus).to_csv('')
        
        return corpus,qualifying_users#{'corpus':corpus, 'qualifying_users':qualifying_users}
        
    def vectorize_corpus(self):
        return self.vectorizer.fit_transform(self.corpus) 
    
    def tsne_reduction(self):
        tsne = TSNE(n_components=2, init = 'random',random_state=42)
        tsne_data = tsne.fit_transform(self.vectorized_corpus)#.toarray())
        return tsne_data

    def auto_elbow_method(data,n_clusters_range:np.linspace,_n_init = 5, _random_state = 42, plot = False):
        """auto elbow method for kmeans

        Args:
            data (_type_): data to clusterize
            n_clusters_range (np.linspace): range of n_clusters for elbow method
            _n_init (int, optional): number of kmeans init. Defaults to 5.
            _random_state (int, optional): for replicability. Defaults to 42.
            plot (bool, optional): plots elbow method. Defaults to False.

        Returns:
            _n_clusters_ (int): returns number of clusters for kmeans 
        """
        # Range of cluster numbers to try
        # Initialize an empty list to store the variance explained by each cluster
        inertia = []

        # Perform K-Means clustering for different values of k
        for n_clusters in tqdm(n_clusters_range,desc='testing clusters in elbow method'):
            kmeans = KMeans(n_clusters=n_clusters,n_init=_n_init,random_state=_random_state)
            kmeans.fit(data)
            inertia.append(kmeans.inertia_)

        if plot:
            # Create the Elbow Method graph
            plt.figure(figsize=(20, 8))
            plt.plot(n_clusters_range, inertia, marker='o')
            plt.title('Elbow Method for Optimal K')
            plt.xlabel('Number of Clusters (K)')
            plt.ylabel('Variance Explained (Inertia)')
            plt.grid(True)
            plt.show()

        variation = [(inertia[i] - inertia[i+1])/ inertia[i] * 100 for i in range(len(inertia)-1)]
        n_clusters = n_clusters_range[variation.index(max(variation)) + 1]
        print(f"El número óptimo de clusters es {n_clusters}")

        return n_clusters

    def launch_kmeans(self,n_clusters:int,data,corpus, plot:bool = False):
        """Launches Kmeans

        Args:
            n_clusters (int): number of clusters
            data (_type_): data to clusterize
            corpus (_type_): original corpus
            plot (bool, optional): plot kmeans. Defaults to False.

        Returns:
            kmeans (Kmeans): Kmeans model
            data_with_clusters (DataFrame): dataframe of corpus with asociated cluster
        """
        # Apply K-Means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_assignments = kmeans.fit_predict(data)
        #cluster_assignments
        # Create a scatter plot with different colors for each cluster

        if plot:
            plt.figure(figsize=(8,8 ))
            sns.scatterplot(x=data[:, 0], y=data[:, 1], hue=cluster_assignments, s=70,marker = '.',palette='hls', legend=False)
            plt.title('User Space Kmeans Clustering Results')
            plt.xlabel('Dimension 1')
            plt.ylabel('Dimension 2')
            
            plt.show()
            #plt.savefig('kmeans_plot.png')
        #print(corpus.shape,cluster_assignments.shape)
        # Create a DataFrame to associate original strings with clusters
        data_with_clusters = pd.DataFrame({'taxnumberprovider':self.qualifying_users,'feature_vector': corpus, 'Cluster': cluster_assignments})
        return kmeans,data_with_clusters
    
    def export_kmeans_model_and_data(self):
        print('Exporting Kmeans model')
        with open(self.save_path+'/kmeans_model.pkl', 'wb') as model_file:
            pickle.dump(self.kmeans_model, model_file)
        self.data_with_clusters.to_csv(self.save_path+'/kmeans_clusters.csv', index=False)

    def export_vectorizer(self):
        file_path = os.path.join(os.getcwd(), 'count_vectorizer_model.pkl')
        print('Exporting vectorizer model')
        # Open the file for writing
        with open(file_path, 'wb') as model_file:
            pickle.dump(self.vectorizer, model_file)
        print('Done')
         

    def export_vectorized_corpus(self):
        self.vectorized_corpus = pd.DataFrame.sparse.from_spmatrix(self.vectorized_corpus)
        self.vectorized_corpus.to_csv(self.save_path+'/vectorized_corpus.csv', index = False)

class RecommenderSystem(UserSpaceGenerator):
    
    def __init__(self, DF:pd.DataFrame, save_path:str = os.getcwd(), autoinitialize = True,autosave = True) -> None:
        self.df = DF
        self.files_in_directory = os.listdir(save_path)
        self.check_files = ['kmeans_clusters.csv','vectorized_corpus.csv','count_vectorizer_model.pkl','kmeans_model.pkl']
        
        is_subset = set(self.check_files).issubset(self.files_in_directory)
        
        if not is_subset:
                super().__init__(DF, autoinitialize = True, autosave=True)
        else:
            with open("kmeans_model.pkl", "rb") as file:
                self.cluster_model = pickle.load(file)
            with open("count_vectorizer_model.pkl", "rb") as file:
                self.vectorizer = pickle.load(file)
            self.vectorized_corpus = pd.read_csv('vectorized_corpus.csv')
            self.data_with_clusters = pd.read_csv('kmeans_clusters.csv')
            
    
        
    def predict(self, taxnumberprovider:str):
        """Predict items (recommend items) to a user based on its taxnumberprovider

        Args:
            taxnumberprovider (str): _description_
        """
        user_vector = UserVector(taxnumberprovider)
        user_vectorized = self.vectorizer.transform([' '.join(user_vector.strings)]).toarray().flatten()
        distances = euclidean_distances(user_vectorized.reshape(1,-1), self.vectorized_corpus)
        nearest_core_point_cluster = self.data_with_clusters['Cluster'][distances.argmin()]
        print(f"Unseen data point belongs to cluster {nearest_core_point_cluster}")
         
        def get_cluster_data(cluster, cluster_method):
            cluster_data = cluster_method[cluster_method['Cluster'] == cluster]
            return cluster_data

        match_df = get_cluster_data(nearest_core_point_cluster,self.data_with_clusters)
        gg = self.df.merge(match_df,on='taxnumberprovider', how='left')
        return gg

In [24]:
UserVector('96.696.000-0')

In [35]:
RS = RecommenderSystem(df)

True


In [36]:
RS_TEST  = RecommenderSystem(df)

True


In [37]:
gg = RS_TEST.predict('77.187.331-6') 
cluster_number= 10
 

Unseen data point belongs to cluster 10
